# Background

The urbansounds dataset proved to be very noisy. A lot of the classes were either irrelevant to our work or too similar for spectrograms to be distinguishable.

We decided to narrow the focus of our modelling efforts by selecting a few categories

This specific model uses three classes collected by the soundflux team

As of March 28th, the classes used are as followes:

- Falling Dummy (simulated human falls from Rescue Randy) from SoundFlux
- General noise from SoundFlux
- Falling Object (semi-bouncy object falling)

Get the data from: https://s3.amazonaws.com/soundflux-urbansounds/curated_soundflux_three_classes.zip

# Loading the data and training the model

In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
set_session(sess)

Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import json
from PIL import Image

In [17]:
def generate_generator_multiple(generator,directories, batch_size, img_height,img_width):
    generators =[]
    for directory in directories:
        gen = generator.flow_from_directory(directory,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=True, 
                                          seed=7)
        print(generator.class_indices)
        generators.append(gen)

    for gen in generators:
        for data, labels in gen:
            yield data, labels

In [4]:
def save_plots(history,target_file_acc,target_file_loss):
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(target_file_acc)
    plt.close()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(target_file_loss)
    plt.close()

In [5]:
def show_plots(history):
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

# Defining relevant parameters

In [6]:
batch_size = 32
img_height=80
img_width = 256
approx_fold_size = 1000

In [7]:
train_folder = "/home/nvidia/curated_soundflux_three_classes/train/"
test_folder = "/home/nvidia/curated_soundflux_three_classes/test/"

In [18]:
datagen = ImageDataGenerator(rescale=1./255,
                            #rotation_range=10,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.2,
                            zoom_range=0.3,
                            #horizontal_flip=True,
                            #vertical_flip=True,
                            fill_mode='nearest')

In [9]:
input_shape = (img_height, img_width,3)
nclass = 3

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers

# Actual Model

In [11]:
base_model = keras.applications.vgg16.VGG16(weights='imagenet', 
                                include_top=False, 
                                input_shape=(img_height, img_width,3))

In [12]:
len(base_model.layers)

19

In [13]:
#base_model.trainable = False
for layer in base_model.layers[:15]:
    layer.trainable = False
"""   #Adding custom Layers
   x = model.output
   x = Flatten()(x)
   x = Dense(4096, activation="relu")(x)
   x = Dropout(0.5)(x)
   x = Dense(4096, activation="relu")(x)
   x = Dropout(0.5)(x)
"""
model = keras.models.Sequential()
model.add(base_model)
model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Dense(512,activation='relu'))
model.add(keras.layers.Dense(64,activation='relu'))
model.add(keras.layers.Dense(32,activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(nclass, activation='softmax'))

In [14]:
opt = optimizers.RMSprop(lr=0.0001,decay=1e-3)
#opt = optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', 
              optimizer=opt,
              metrics=['accuracy'])
#needed to reset weigh"""ts!
model.save_weights('raw_model.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 2, 8, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 99        
Total para

# Data Generator

In [23]:
train_generator = datagen.flow_from_directory(train_folder,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=True, 
                                          seed=7)

test_generator = datagen.flow_from_directory(test_folder,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=True, 
                                          seed=7)


Found 291 images belonging to 3 classes.
Found 137 images belonging to 3 classes.


In [24]:
train_generator.class_indices

{'falling_dummy': 0, 'falling_object': 1, 'general_noise': 2}

In [23]:
#RESET WEIGHTS!!
#model.load_weights('raw_model.h5')
#
history = model.fit_generator(train_generator,
                          steps_per_epoch=approx_fold_size/batch_size,
                          validation_data = test_generator,
                          validation_steps = approx_fold_size/batch_size,
                          epochs=20,
                          shuffle=True, 
                          verbose=True)

Epoch 1/20
Found 291 images belonging to 3 classes.
32/31 [==============================] - 73s 2s/step - loss: 0.7704 - acc: 0.6477 - val_loss: 0.4692 - val_acc: 0.7946
Epoch 2/20
32/31 [==============================] - 49s 2s/step - loss: 0.4353 - acc: 0.8338 - val_loss: 0.3396 - val_acc: 0.8760
Epoch 3/20
32/31 [==============================] - 48s 2s/step - loss: 0.3305 - acc: 0.8802 - val_loss: 0.3213 - val_acc: 0.8725
Epoch 4/20
32/31 [==============================] - 48s 2s/step - loss: 0.2718 - acc: 0.9086 - val_loss: 0.3139 - val_acc: 0.8899
Epoch 5/20
32/31 [==============================] - 48s 2s/step - loss: 0.2589 - acc: 0.9111 - val_loss: 0.2807 - val_acc: 0.8950
Epoch 6/20
32/31 [==============================] - 49s 2s/step - loss: 0.2199 - acc: 0.9298 - val_loss: 0.4367 - val_acc: 0.8725
Epoch 7/20
32/31 [==============================] - 48s 2s/step - loss: 0.2115 - acc: 0.9429 - val_loss: 0.2966 - val_acc: 0.9061
Epoch 8/20
32/31 [==============================]

Best so far -> lr = 0.0001, decay=1e-3, 13 epochs. Filal val accuracy = 92, loss = 0.4309

# Test on the final model

In [24]:
test_generator = generate_generator_multiple(generator=datagen,
                                           directories = [test_folder],
                                           batch_size=batch_size,
                                           img_height=img_height,
                                           img_width=img_width)
model.evaluate_generator(test_generator,
                              steps=approx_fold_size/batch_size,
                              verbose=True)

Found 137 images belonging to 3 classes.
32/31 [==============================] - 23s 730ms/step


[0.21583733042128148, 0.9401805873783663]

In [25]:
model.save_weights('curated_model_three_classes_unfrozen_layers_v1.h5')

In [26]:
model_json = model.to_json()
with open("curated_model_three_classes_unfrozen_layers_v1.json", "w") as json_file:
    json_file.write(model_json)

In [30]:
with open("curated_model_three_classes_unfrozen_layers_v1_class_indices.json", "w") as json_file:
    train_generator.class_indices
    json_file.write(json.dumps(train_generator.class_indices))

The model weights were saved on the very first run of this model and ara available here: 
https://s3.amazonaws.com/soundflux-urbansounds/curated_model_three_classes_unfrozen_layers_v1.zip